Federated Learning (FL) is a distributed machine learning process in which each participant node (or party) retains their data locally and interacts with other participants via a learning protocol. One main driver behind FL is the need to not share data with others due to privacy and confidentially concerns. Another driver is to improve the speed of training a machine learning model by leveraging other participants' training processes.

Setting up such a federated learning system requires setting up a communication infrastructure, converting machine learning algorithms to federated settings and in some cases knowing about the intricacies of security and privacy enabling techniques such as differential privacy and multi-party computation.

#TensorFlow Federated: Machine Learning on Decentralized Data

##Install necessary Libraries

TensorFlow Federated (TFF) is an open-source framework for machine learning and other computations on decentralized data. 

Federated Learning: Collaborative Machine Learning without Centralized Training Data.

Standard machine learning approaches require centralizing the training data on one machine or in a datacenter. And Google has built one of the most secure and robust cloud infrastructures for processing this data to make our services better. Now for models trained from user interaction with mobile devices, Introducing an additional approach: Federated Learning.

In [ ]:
pip install --upgrade tensorflow-federated

     |████████████████████████████████| 604kB 27.3MB/s 
     |████████████████████████████████| 194kB 52.2MB/s 
     |████████████████████████████████| 174kB 35.9MB/s 
     |████████████████████████████████| 51kB 8.3MB/s 
ERROR: spacy 2.2.4 has requirement tqdm<5.0.0,>=4.38.0, but you'll have tqdm 4.28.1 which is incompatible.
ERROR: pymc3 3.11.2 has requirement cachetools>=4.2.1, but you'll have cachetools 3.1.1 which is incompatible.
ERROR: fbprophet 0.7.1 has requirement tqdm>=4.36.1, but you'll have tqdm 4.28.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: attrs 21.2.0
    Uninstalling attrs-21.2.0:
      Successfully uninstalled attrs-21.2.0
  Found existing installation: cachetools 4.2.2
    Uninstalling cachetools-4.2.2:
      Successfully uninstalled cachetools-4.2.2
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully unins

#Import Libraries

In [ ]:
import tensorflow as tf
import tensorflow_federated as tff

In [ ]:
# Load simulation data.
source, _ = tff.simulation.datasets.emnist.load_data()
def client_data(n):
  return source.create_tf_dataset_for_client(source.client_ids[n]).map(
      lambda e: (tf.reshape(e['pixels'], [-1]), e['label'])
  ).repeat(10).batch(20)

# Pick a subset of client devices to participate in training.
train_data = [client_data(n) for n in range(3)]

# Wrap a Keras model for use with TFF.
def model_fn():
  model = tf.keras.models.Sequential([
      tf.keras.layers.Dense(10, tf.nn.softmax, input_shape=(784,),
                            kernel_initializer='zeros')
  ])
  return tff.learning.from_keras_model(
      model,
      input_spec=train_data[0].element_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

# Simulate a few rounds of training with the selected client devices.
trainer = tff.learning.build_federated_averaging_process(
  model_fn,
  client_optimizer_fn=lambda: tf.keras.optimizers.SGD(0.1))

#for avoid (Cannot run the event loop while another loop is running)

In [ ]:
!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()

In [ ]:
state = trainer.initialize()

In [ ]:
for _ in range(5):
  state, metrics = trainer.next(state, train_data)
  print(metrics['train']['loss'])

12.946636
13.406321
12.45098
12.208817
11.7869
